# Web Search 2018 - Tutorial 4: Graph method - Label Propagation
## Contents

1. [Overview](#head1)
  1. [Code Imports](#head11)
2. [Iterative Label Propagation](#head2)
  1. [Dataset - MNIST Digits](#head21)
  2. [Parameters Initialization](#head22)
  3. [Algorithm Implementation](#head23)
  4. [Evaluation](#head24)
  5. [Parameter Tuning](#head25)
3. [Iterative Label Propagation over different Search Spaces](#head3)


## <a name="head1"></a> Overview

Despite the fact that large amounts of data are available in the Web domain, only a small set of such data may be categorized.
The goal of this lab is to understand how one can leverage on a reduced set of categorized data, to enrich sets of uncategorized data. 

Specifically, in this lab you will be using the Label Propagation (LP) algorithm, which consists of a semi-supervised graph approach to annotate uncategorized/unlabelled data starting from a small set of categorized/labelled data.

**Lab objectives:**
* Implement the iterative version of the Label Propagation algorithm and apply it to the MNIST digits dataset
* Evaluate the performance of the Iterative LP algorithm;
* Create multiple graphs, one per feature space (visual and textual), and apply the Iterative LP algorithm.

### <a name="head11"></a> Code Imports

In [1]:
import numpy as np
from numpy.random import shuffle

# <a name="head2"></a> Iterative Label Propagation

Consider a dataset $X=\{x_1, x_2, \ldots, x_L, \ \ x_{L+1}, \ldots, x_N\}$, with $N$ data points, where each $x_i$ consists of some feature representation of document $i$. Given a categories set $C=\{1, 2, \ldots, |C|\}$, it is assumed that the first $L$ data points are labelled with a label $c \in C$, and the remaining ones are unlabelled.

You are asked to implement the Iterative Label Propagation algorithm. Please refer to the "Mining Data Graphs" class (lectured on 29/10), namely slides 28, 29 and 30, for a description of the algorithm steps.

For more information, you can check the original paper: Dengyong Zhou, Olivier Bousquet, Thomas Navin Lal, Jason Weston, Bernhard Schoelkopf. Learning with local and global consistency (2004) http://citeseer.ist.psu.edu/viewdoc/summary?doi=10.1.1.115.3219


## <a name="head21"></a> Dataset - MNIST Digits

Every time you implement an algorithm, it is good practice to debug your implementation on a simple dataset. For this purpose, you should first use the MNIST Digits datasets. This dataset is widely used as a baseline to evaluate machine learning algorithms. You can learn more about it in https://en.wikipedia.org/wiki/MNIST_database.

In [2]:
# Load mnist digits dataset using sklearn. Every digit image is represented as a 8x8 (64) RGB image.

from sklearn import datasets
mnist_digits = datasets.load_digits()
print("MNIST digits images shape: {}".format(mnist_digits.images[0].shape))
print("MNIST digits dataset shape: {}".format(mnist_digits.data.shape))
print("MNIST digits categories shape: {} - Categories C: {}".format(mnist_digits.target.shape, set(mnist_digits.target)))

MNIST digits images shape: (8, 8)
MNIST digits dataset shape: (1797, 64)
MNIST digits categories shape: (1797,) - Categories C: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [3]:
indices = np.arange(len(mnist_digits.data))

# Shuffle the array - Modifies the array inplace 
shuffle(indices)

X = mnist_digits.data[indices]
y_target = mnist_digits.target[indices]

total_images = X.shape[0]

## <a name="head22"></a>  Parameters Initialization

In [4]:
# Let's assume that 20% of the dataset is labeled
labeled_set_size = int(total_images*0.2)

# You should tune these values
alpha = 1
num_iterations = 500
sigma = 1

In [5]:
indices_labeled = indices[:labeled_set_size]
indices_unlabeled = indices[labeled_set_size:]

print("Total images labeled: {} - Total images unlabeled: {}".format(len(indices_labeled), len(indices_unlabeled)))

Total images labeled: 359 - Total images unlabeled: 1438


In [6]:
# Convert labels to a one-hot-encoded vector
from keras.utils import to_categorical

# Keep groundtruth labels
Y_true = to_categorical(y_target)

Y = to_categorical(y_target)
print(Y.shape)

# Remove labels of "unlabeled" data
Y[indices_unlabeled,:] = np.zeros(Y.shape[1])

Using TensorFlow backend.


(1797, 10)


## <a name="head23"></a>  Algorithm Implementation

In [7]:
# Step 1 - Extract features for each image (HoG/CNN/HoC) in X
import numpy as np
from numpy import linalg as LA
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.io import imread
from sklearn.preprocessing import normalize
from skimage import color
from skimage import data, exposure

feature = "hog"

if feature is "hog":
    pixels_per_cell=(2,2)
    orientations=8
    X = []
    for img in mnist_digits.images:
        hist, hog_img = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, visualize=True, block_norm='L2-Hys')
        hist = np.squeeze(normalize(hist.reshape(1, -1), norm="l2"))
        X.append(hist)
    X = np.array(X)
    print("Shape of feature matrix: {}".format(X.shape))
else:
    print("No feature selected")

# Step 2 - Initialize matrix F
F = Y

# Step 3 - Compute matrix W
from sklearn.metrics.pairwise import euclidean_distances
W = np.exp(-1*euclidean_distances(X, X)/(2*sigma**2))

# Step 4 - Normalize W
D = np.zeros(W.shape)
np.fill_diagonal(D, W.sum(axis=0))

D12 = np.zeros(D.shape)
from numpy.linalg import inv
D12 = inv(np.sqrt(D))

S = np.dot(D12, W)
S = np.dot(S, D12)

# Step 5 - Perform the F update step num_iterations steps
for i in range(1, num_iterations):
    T1 = alpha * S
    T1 = np.dot(T1,F)
    T2 = 1 * alpha * Y_true
    F = T1 + T2
F = np.argmax(F, axis=1)
Y = to_categorical(F)

Shape of feature matrix: (1797, 288)


## <a name="head24"></a>  Evaluation

Evaluate the results of each run of the Iterative LP.

In [8]:
# The function classification_report computes and prints a set of commonly used metrics.
# docs: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report
from sklearn.metrics import classification_report
np.set_printoptions(threshold=np.nan)

# Get the predictions of the unlabeled documents
Y_pred = Y[indices_unlabeled, :]
#print(Y_pred)

# Get the corresponding groundtruth
y_gt = Y_true[indices_unlabeled, :]
#print(y_gt)

print(classification_report(y_gt, Y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       154
          1       1.00      1.00      1.00       142
          2       0.00      0.00      0.00       141
          3       0.20      1.00      0.34       143
          4       1.00      1.00      1.00       150
          5       1.00      1.00      1.00       145
          6       1.00      1.00      1.00       146
          7       0.00      0.00      0.00       135
          8       0.00      0.00      0.00       136
          9       1.00      1.00      1.00       146

avg / total       0.53      0.61      0.54      1438



/Users/franciscorodrigues/anaconda3/envs/websearch/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## <a name="head25"></a> Parameter Tuning
Assess the impact of each of the algorithm parameters.

# <a name="head3"></a> Iterative Label Propagation over different Search Spaces

Re-run your Iterative LP implementation, for each of the previously implemented search spaces (HoG, CNN and HoC).

Evaluate and discuss the obtained results.